In [2]:
from time import time

import torch
import torch.nn.functional as F
import torchvision
from cnn_net import ConvNet
from torch import nn, optim
from torch.utils.data import DataLoader

from alphafed.fed_avg import FedAvgScheduler


class DemoAvg(FedAvgScheduler):

    def __init__(self,
                 max_rounds: int = 0,
                 merge_epochs: int = 1,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0,
                 involve_aggregator: bool = False,
                 batch_size: int = 128,
                 learning_rate: float = 0.01,
                 momentum: float = 0.9) -> None:
        """初始化参数说明.

        以下为 `FedAvgScheduler` 父类定义的初始化参数。
        max_rounds:
            训练多少轮。
        merge_epochs:
            每次参数聚合前，在本地训练几个 epoch。
        calculation_timeout:
            本地训练超时时间。
        log_rounds:
            每隔几轮训练执行一次测试，评估记录当前训练效果。
        involve_aggregator:
            聚合方是否也使用本地数据参与训练，默认不参与，只负责聚合。

        以下 `DemoAvg` 自定义的扩展初始化参数。
        batch_size、learning_rate、momentum
            训练参数。
        """
        super().__init__(max_rounds=max_rounds,
                         merge_epochs=merge_epochs,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds,
                         involve_aggregator=involve_aggregator)
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.momentum = momentum

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.seed = 42
        torch.manual_seed(self.seed)

    def build_model(self) -> nn.Module:  # 返回模型实例
        model = ConvNet()
        return model.to(self.device)

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:  # 返回优化器实例
        return optim.SGD(model.parameters(),
                         lr=self.learning_rate,
                         momentum=self.momentum)

    # 网络下载 MNIST 训练数据用于训练，如果是实际使用场景，需要开发者自行处理数据加载
    def build_train_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=True
        )

    # 本示例不使用验证集数据，所以不需要实现 build_validation_dataloader

    # 网络下载 MNIST 测试数据用于测试，如果是实际使用场景，需要开发者自行处理数据加载
    def build_test_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=False
        )

    def train_an_epoch(self) -> None:  # 执行一个 epoch 训练的逻辑，与本地训练模型时的代码相同
        self.model.train()
        for data, labels in self.train_loader:
            data, labels = data.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, labels)
            loss.backward()
            self.optimizer.step()

    def run_test(self):  # 执行一次测试的逻辑，与本地模型测时的代码相同
        self.model.eval()
        start = time()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                data, labels = data.to(self.device), labels.to(self.device)
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        # 记录 TensorBoard 日志，self.current_round 的值为“当前是训练的第几轮”
        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

In [3]:
from alphafed import mock_context

# 聚合方的模拟启动脚本
scheduler = DemoAvg(
    max_rounds=3,  # 受本地资源限制，运行速度可能会很慢，调试时不建议设置太高
    log_rounds=1,
    calculation_timeout=1800  # 受本地资源限制，运行速度可能会很慢，设置太低容易导致超时
)

task_id = 'cbb3ffd0-838c-41ca-a41a-7c11cae29181'  # 设置一个假想 ID
# 算法实际运行时会从任务管理器获取任务参与节点的 Node ID 列表，但是在模拟环境不能通过
# 访问实际接口获得这个信息，所以需要通过 nodes 参数将这个列表配置在模拟环境中。
aggregator_id = '1bb9feba-7b53-455b-b127-0eb19ffc9d3f'  # 设置一个假想 ID
col_id_1 = '663ad4b0-b617-409f-8bc9-3682b30f7f30'  # 设置一个假想 ID
col_id_2 = '0fc1a571-2920-47bf-9e4e-b4edb7fa2caa'  # 设置一个假想 ID
with mock_context(id=aggregator_id, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)

2023-02-03 08:34:38,456|DEBUG|scheduler|_switch_status|125:
self.status='init'
2023-02-03 08:34:38,480|INFO|scheduler|push_log|118:
Begin to validate local context.
2023-02-03 08:34:38,480|INFO|scheduler|push_log|118:
Local context is ready.
2023-02-03 08:34:38,482|INFO|scheduler|push_log|118:
Node 1bb9feba-7b53-455b-b127-0eb19ffc9d3f is up.
2023-02-03 08:34:38,482|DEBUG|scheduler|_switch_status|125:
self.status='gethering'
2023-02-03 08:34:38,483|INFO|scheduler|push_log|118:
Waiting for participants taking part in ...
2023-02-03 08:34:38,483|DEBUG|fed_avg|_wait_for_gathering|432:
_wait_for_gathering ...
2023-02-03 08:35:02,523|INFO|scheduler|push_log|118:
Welcome a new participant ID: 663ad4b0-b617-409f-8bc9-3682b30f7f30.
2023-02-03 08:35:02,523|INFO|scheduler|push_log|118:
There are 2 participants now.
2023-02-03 08:35:17,572|INFO|scheduler|push_log|118:
Welcome a new participant ID: 0fc1a571-2920-47bf-9e4e-b4edb7fa2caa.
2023-02-03 08:35:17,573|INFO|scheduler|push_log|118:
There are 